In [1]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader


from dotenv import load_dotenv
load_dotenv()

True

In [68]:
# PDF Load
# pdf_file_path = "../streamlit_pdf_web/visa_kor.pdf"
# loader = PDFPlumberLoader(pdf_file_path)
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
# docs = text_splitter.split_documents(docs)

from visa_kor_docs import VisaDocuments

docs = VisaDocuments


In [3]:
len(docs)

20

In [64]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [66]:
# print(len(doc_ids))
doc_ids

['b4e25cc7-3a6e-474f-9db0-37ec49880349',
 '3faa37a6-a1d3-40b8-b9b0-ba1355fbb4fd',
 'abdc2802-1d1f-4228-b901-6806cc1efe58',
 '70896c2d-988b-4e01-9e16-2741854ecde8',
 '17def707-db8b-4e0f-88e4-7b580f544de2',
 '03e84b57-5afe-4d04-843f-a07c0ccd81cd',
 'b384658f-5c62-41d6-9cd9-19427e3e4bc5',
 '828bdb89-b63c-44d9-98f5-f234f637ea47',
 '5f82d73c-408b-4b27-af10-993e98b71311',
 '000b291b-a861-4451-968c-5e02b834aab3',
 'f567c12c-c1c1-414b-bcb2-98f27bb21025',
 '72afb685-5878-4484-bdc1-8c933826fc72',
 '86935c3b-3d09-48b3-8867-b72754971e4e',
 '251b04c0-04c2-42f8-904d-548e8cf9be6c',
 '84e77fa3-04e0-4b84-b66b-98fa5e85847b',
 '194565d5-8fa2-4e13-bccc-c035126e0cef',
 '3d0ddf9b-f4d6-4b81-89a6-44203bbd6b64',
 '9e87e822-d4cf-4286-8c3c-4b205b50adf4',
 '153bb66e-adc4-45e2-82ef-4046e5f07150',
 '7ac527fb-8e0c-491d-a9c9-29fe7871be75']

In [47]:
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [42]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [43]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [45]:
from langchain.retrievers.multi_vector import SearchType

# retriever.search_type = SearchType.mmr

related_docs = retriever.vectorstore.similarity_search("비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?")

pretty_print_docs(related_docs)

Document 1:

# E-9: 고용허가제 비전문취업 비자
----------------------------------------------------------------------------------------------------
Document 2:

# E-9: 고용허가제 비전문취업 비자
----------------------------------------------------------------------------------------------------
Document 3:

○ 체류자격: E-9-3
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 농업: 작물재배업
                ■ 축산업
                ■ 농축산서비스업: 작물재배 및 축산관련 서비스업
        ● 어업
            ○ 체류자격: E-9-4
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 연근해어업: 연안어업ᆞ근해어업
                    ● 선원법 적용을 받지 않는 20톤 미만 어선 종사자에 한함
                ■ 양식어업
                ■ 소금채취업
        ● 임업
----------------------------------------------------------------------------------------------------
Document 4:

○ 체류자격: E-9-3
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 농업: 작물재배업
                ■ 축산업
                ■ 농축산서비스업: 작물재배 및 축산관련 서비스업
        ● 어업
            ○ 체류자격: E-9-4
            ○ 적용범위(E-9 근로자를 고용할 수 있

In [10]:
# len(retriever.get_relevant_documents("비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?")[0].page_content)

from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr

related_docs = retriever.get_relevant_documents("비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?")
pretty_print_docs(related_docs)

Document 1:


        # E-9: 고용허가제 비전문취업 비자

        고용허가제
            고용허가제 해당자와 활동범위
        고용허가제 해당자에 대한 사증 발급
            사증발급 후 1회 체류기간의 상한
            사증발급 허용업종 및 체류자격 약호(기호): E-9 근로자를 채용할 수 있는 업종과 범위
            사증발급인정서를 받아야만 E-9 사증을 발급받을 수 있음
            2012.8.1부터 범죄경력증명서 및 건강상태확인서 제출
            비전문취업(E-9) 자격 사증발급인정서 발급 절차
            재입국특례 제도(구 성실근로자 제도)
            재입국특례자에 대한 사증 신청 및 발급 방법
            재입국특례자에 대한 우대 내용
            고용허가제 해당자의 근무처(직장)의 변경
            고용허가제 해당자의 근무처(직장)를 변경할 수 있는 조건
            고용허가제 해당자의 근무처(직장)를 변경하는 절차 및 제출 서류
            고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
            고용허가제 해당자의 체류자격 변경허가 - 사증 변경
            고용허가제 해당자의 체류기간 연장 허가 - 사증 연장
            고용허가제 해당자의 재입국허가
            고용허가제 해당자의 외국인등록
            고용허가제 해당자의 고용변동 신고
        
----------------------------------------------------------------------------------------------------
Document 2:

## 사증발급인정서를 받아야만 E-9 사증을 발급 받을수있음
        ● 비전문취업(E-9) 사증은 출입국ᆞ외국인청(사무소ᆞ출장소)장이 발급한 사증발급인정서에 의해서만 사증을

In [11]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [70]:
# ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}, You MUST answer in Korean.")

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("이 문서는 한국의 비자와 관련된 문서입니다. 다음의 주어진 문서를 보고 요약해주세요:\n\n{doc}")
    | ChatOpenAI(max_retries=0, model="gpt-4-1106-preview")
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 5})

In [71]:
summaries

['이 문서는 한국의 E-9 비자, 즉 비전문취업 비자에 대한 내용을 담고 있습니다. E-9 비자는 고용허가제 하에 일정 업종에서 일할 수 있는 외국인 근로자들을 위한 체류자격입니다. 주요 내용은 다음과 같습니다:\n\n- E-9 비자는 고용허가제를 통해 특정 업종에서 일할 수 있는 외국인근로자들에게 발급됩니다.\n- 비자를 받기 위해서는 사증발급인정서를 받아야 하며, 체류 기간은 1회에 한정됩니다.\n- 2012년 8월 1일부터는 범죄경력증명서와 건강상태확인서를 제출해야 합니다.\n- 비전문취업(E-9) 자격을 위한 사증발급인정서 발급 절차가 명시되어 있습니다.\n- 재입국 특례 제도(구 성실근로자 제도)를 통해 성실하게 일한 근로자는 재입국 시 일정 우대를 받을 수 있습니다.\n- 근로자가 근무처(직장)를 변경하고자 할 때 필요한 조건, 절차 및 제출 서류가 안내되어 있습니다.\n- 농업 분야에서 일하는 외국인근로자의 경우 근무처 변경에 관한 추가 정보가 제공됩니다.\n- E-9 비자 소지자가 체류자격을 변경하거나 체류 기간을 연장하고자 할 때 필요한 절차가 설명되어 있습니다.\n- 외국인등록, 재입국허가, 고용변동 신고 등 관련 행정 절차에 대한 정보도 포함되어 있습니다.\n\n이 문서는 E-9 비자 소지자들이 한국에서의 취업과 체류에 필요한 정보와 지침을 제공하며, 비자의 발급 및 관리, 체류 조건과 관련된 세부 사항을 다루고 있습니다.',
 '이 문서는 한국의 고용허가제에 대해 설명하고 있습니다.\n\n고용허가제는 외국인근로자의 고용에 대한 법적 틀을 제공하는 인력제도입니다. 이 제도는 외국인 근로자에게 특정 사업주에 의한 고용을 조건으로 최대 4년 10개월간의 취업을 가능하게 합니다. 2004년 8월에 시행된 이 제도는 현재까지 16개국과의 양해각서(MOU)를 통해 운영되고 있습니다.\n\n고용허가는 상시근로자가 300인 미만이거나 자본금이 80억원 이하인 중소기업에 대해서만 외국인 근로자의 고용을 허가합니다. 고용허가제에 참여하는 선정 국가는 태국

In [14]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [50]:
len(doc_ids)

20

In [15]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [16]:
sub_docs = vectorstore.similarity_search("비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?")

In [17]:
pretty_print_docs(sub_docs)

Document 1:

E-9 근로자를 채용할 수 있는 업종과 범위에 대한 내용을 다루고 있습니다. 제조업, 건설업, 농축산업, 어업, 임업, 광업, 서비스업 등 다양한 업종에 대한 정보가 포함되어 있습니다. 각 업종별로 E-9 근로자를 고용할 수 있는 조건과 적용 범위가 설명되어 있으며, 추가로 한식 음식점업, 호텔‧숙박업, 임업, 광업이 24년도 이후에 사증발급인정서 발급 신청이 가능하다는 내용도 포함되어 있습니다.
----------------------------------------------------------------------------------------------------
Document 2:

E-9 비전문취업 비자는 외국인 근로자가 한국에서 일할 수 있도록 하는 고용허가제입니다. 이 비자를 받기 위해서는 사증 발급 및 변경, 체류기간 연장, 재입국허가 등의 절차를 따라야 합니다. 또한, 범죄경력증명서와 건강상태확인서 제출이 필요하며, 해당자의 근무처 변경 및 고용변동 신고도 필요합니다. 이러한 고용허가제는 비전문취업자를 위한 제도이며, 해당자의 활동 범위와 사증 발급에 관한 규정이 상세히 설명되어 있습니다.
----------------------------------------------------------------------------------------------------
Document 3:

E-9 비전문취업 사증을 받기 위해서는 사증발급인정서를 받아야만 가능하다. 출입국ᆞ외국인청(사무소ᆞ출장소)장이 발급한 사증발급인정서에 의해서만 사증을 발급할 수 있다.
----------------------------------------------------------------------------------------------------
Document 4:

고용허가제 해당자의 체류자격 변경허가를 받기 위한 기타자격(G-1) 사증 소지자의 비전문취업(E-9) 자격 회복절차에 대해 설명하고 있습니다. 대상자는 고용허가제

In [18]:
retrieved_docs = retriever.get_relevant_documents("비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?")
pretty_print_docs(retrieved_docs)

Document 1:

## 사증발급 허용업종 및 체류자격 약호(기호): E-9 근로자를 채용할 수 있는 업종과 범위
        ● 제조업: 제조업, 뿌리산업
            ○ 체류자격: E-9-1
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 상시근로자 300인 미만 또는 자본금 80억원 이하 제조업
                ■ 단, 상기 기준에 충족하지 않더라도 아래 증빙서류 제출 시 인정 : 중소기업(지방중소기업청에서 발급한 ‘중소기업확인서’) / 비수도권 소재 
                뿌리산업 중견기업 (한국생산기술연구원에서 발급한 ‘뿌리기업 확인서’ 및 한국중견기업연합회에서 발급한 ‘중견기업 확인서’)
        ● 건설업
            ○ 체류자격: E-9-2
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 모든 건설공사
                ■ ※ 발전소·제철소·석유화학 건설현장의 건설업체 중 건설면허가 산업환경 설비인 경우에는 적용 제외
        ● 농축산업
            ○ 체류자격: E-9-3
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 농업: 작물재배업
                ■ 축산업
                ■ 농축산서비스업: 작물재배 및 축산관련 서비스업
        ● 어업
            ○ 체류자격: E-9-4
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 연근해어업: 연안어업ᆞ근해어업
                    ● 선원법 적용을 받지 않는 20톤 미만 어선 종사자에 한함
                ■ 양식어업
                ■ 소금채취업
        ● 임업
            ○ 체류자격:
      

In [19]:
len(retrieved_docs[0].page_content)

3035

In [20]:
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model="gpt-4-0613", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", # stuff, map_reduce, refine, map_rerank, 
    # retriever=ensemble_retriever,
    retriever=retriever,
    return_source_documents=True
)

In [21]:
result = qa_chain("비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?")

/Users/heewungsong/anaconda3/envs/pyautogen/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [22]:
print(result["result"])

비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 다음과 같습니다:

1. 제조업: 체류자격 코드 E-9-1
2. 건설업: 체류자격 코드 E-9-2
3. 농축산업: 체류자격 코드 E-9-3
4. 어업: 체류자격 코드 E-9-4
5. 임업: 체류자격 코드가 명시되어 있지 않습니다.
6. 광업: 체류자격 코드가 명시되어 있지 않습니다.
7. 서비스업: 체류자격 코드 E-9-5

각 업종에는 특정 조건이 적용되며, 이는 위에서 제공된 정보를 참조하시면 됩니다.


In [23]:
result = qa_chain(" 체류기간 연장 허가를 받기 위해서는 어떤 서류를 제출해야 하나요?")

In [24]:
print(result["result"])

체류기간 연장 허가를 받기 위해 제출해야 하는 서류는 다음과 같습니다:

1. 신청서(별지 34호 서식)
2. 여권 및 외국인등록증
3. 수수료
4. 고용허가서 사본
5. 표준근로계약서 사본
6. 사업자등록증 사본
7. 입국 후 3년 만료 재고용에 따른 최대 1년 10개월 추가 연장의 경우 ‘취업기간만료자 취업활동기간 연장확인서(고용노동부 발급)’
8. 체류지 입증서류(임대차계약서, 숙소제공 확인서, 체류기간 만료예고 통지우편물, 공공요금 납부영수증, 기숙사비 영수증 등)


In [25]:
result = qa_chain("고용주가 외국인 근로자의 고용변동 사항을 신고하지 않을 경우 어떤 조치가 있나요?")

In [26]:
print(result["result"])

문서에는 고용주가 외국인 근로자의 고용변동 사항을 신고하지 않을 경우에 대한 구체적인 조치에 대한 정보가 제공되지 않았습니다. 따라서 그에 대한 답변을 제공할 수 없습니다.


In [27]:
result = qa_chain("비자 연장을 위해 제출해야 하는 건강검진 항목은 무엇인가요?")
print(result["result"])

죄송합니다, 제공된 정보 중에서 비자 연장을 위해 제출해야 하는 건강검진 항목에 대한 정보는 포함되어 있지 않습니다.


In [29]:
bad_question_list = [
    "비자 연장을 위해 제출해야 하는 건강검진 항목은 무엇인가요?",
    "체류기간 연장을 위한 수수료는 얼마인가요?",
    "E-9 비자의 주요 목적은 무엇인가요?",
    "고용변동 신고의 중요성은 무엇인가요?",
    "외국인 근로자의 권리와 의무에 대한 설명은 어디서 찾을 수 있나요?",
    "한국에서의 건강검진 기준은 외국인 근로자에게 어떤 영향을 미치나요?",
    "업체별 외국인 근로자 허용인원은 어떤 기준으로 정해지나요?",
    "고용주가 외국인 근로자의 고용변동 사항을 신고하지 않을 경우 어떤 조치가 있나요?",
    "건설업 현장 외국인력 고용 시 건설 면허가 산업환경설비인 경우 제외되는 이유는 무엇인가요?",
]


for q in bad_question_list:
    result = qa_chain(q)
    print(f"{result['result']}\n\n")

죄송합니다, 제공된 정보 중에서 비자 연장을 위해 제출해야 하는 건강검진 항목에 대한 정보는 포함되어 있지 않습니다.


문서에 체류기간 연장을 위한 수수료에 대한 구체적인 정보가 제공되지 않았습니다.


E-9 비자는 고용허가제 비전문취업 비자로, 특정 업종에서 근로자를 채용할 수 있는 비자입니다. 이 비자는 제조업, 건설업, 농축산업, 어업, 임업, 광업, 서비스업 등 다양한 업종에서 외국인 근로자를 고용하는 데 사용됩니다.


고용변동 신고는 외국인근로자의 고용 상태에 대한 정확한 정보를 유지하고 관리하는 데 중요합니다. 이는 외국인근로자의 권리를 보호하고, 고용주의 책임을 명확히 하는데 필요한 절차입니다. 또한, 이를 통해 정부는 외국인근로자의 고용 상황을 정확히 파악하고, 이에 따른 적절한 정책을 수립하고 실행할 수 있습니다. 고용변동을 신고하지 않을 경우, 법적인 문제가 발생할 수 있으므로, 고용주는 이를 반드시 준수해야 합니다.


죄송합니다, 제공된 정보에는 외국인 근로자의 권리와 의무에 대한 정보를 찾을 수 있는 곳에 대한 정보가 없습니다.


죄송합니다, 제공된 정보에는 한국에서의 건강검진 기준이 외국인 근로자에게 미치는 영향에 대한 내용이 포함되어 있지 않습니다.


외국인 근로자의 허용인원은 업종별·사업장별 고용허용 기준에 의해 정해집니다. 이는 사업장의 규모, 업종, 필요한 인력 등을 고려하여 결정됩니다. 그러나 사업장별 신규 고용한도의 제한을 받지 않더라도, 총 고용허용인원을 초과할 수는 없습니다.


문서에는 고용주가 외국인 근로자의 고용변동 사항을 신고하지 않을 경우에 대한 구체적인 조치에 대한 정보가 제공되지 않았습니다.


제공된 정보 내에서는 건설업 현장 외국인력 고용 시 건설 면허가 산업환경설비가 제외되는 이유에 대한 정보를 찾을 수 없습니다.




In [72]:
good_question_list = [
    "E-9 비자 신청 시 필요한 서류는 무엇인가요?",
    "고용허가서 신청 절차는 어떻게 되나요?",
    "재입국 특례 제도 신청을 위한 조건은 무엇인가요?",
    "비전문취업 비자의 일반적인 허용 업종 범위는 어디까지인가요?",
    "고용허가제 적용 대상 국가는 어떤 나라들인가요?",
    "비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?",
    "사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?",
    "범죄경력증명서와 건강상태확인서는 어떤 요건을 갖추어야 하나요?",
    "재입국 특례제도의 요건과 절차는 어떻게 되나요?",
    "근무처 변경 시 제출해야 하는 서류들은 무엇인가요?",
    "농업분야에서 근무처를 추가로 가질 수 있는 절차는 어떻게 되나요?",
    "체류기간 연장 허가를 받기 위해서는 어떤 서류를 제출해야 하나요?",
    "고용허가제 해당 업종 중 서비스업에서 E-9 비자로 취업할 수 있는 세부 업종과 조건은 무엇인가요?",
    "건설업 현장에서 동일 원청 내 다른 업체로 인력 이동 시 사업장 변경 횟수 제한에 포함되지 않는 이유와 절차는 어떻게 되나요?",
    "재입국 특례제도 신청 시 고용주와 1년 이상의 근로계약을 맺어야 하는 이유는 무엇인가요?",
    "체류기간 연장 허가 신청 시 구직 등록 유효기간 이전에 체류기간 만료일이 도래하면 어떤 특례를 받을 수 있나요?",
    "기타(G-1) 체류자격에서 비전문취업(E-9) 자격으로 변경할 때의 절차와 조건은 어떻게 되나요?",
    "농업 분야에서 근무처를 추가로 가질 경우 원 근무처와 추가 근무처 고용주의 신고의무는 어떻게 달라지나요?",
    "임업과 광업 분야에서 E-9 비자 발급 시 적용되는 사업장 조건은 무엇인가요?",
    "외국인등록 시 제출해야 하는 마약검사확인서의 구체적인 기준과 유의사항은 무엇인가요?",
    "재입국 특례제도를 이용할 경우 입국 전 취업교육이 면제되는 이유는 무엇인가요?",
]


for q in good_question_list:
    result = qa_chain(q)
    print(f"{q}\n")
    print(f"{result['result']}\n\n")

E-9 비자를 신청하기 위해서는 출입국·외국인청(사무소·출장소)장이 발급한 사증발급인정서가 필요합니다. 이외에도 2012년 8월 1일부터 범죄경력증명서 및 건강상태확인서 제출이 필요하게 되었습니다. 이러한 서류들을 통해 비전문취업(E-9) 자격 사증발급인정서 발급 절차를 진행할 수 있습니다.


죄송합니다, 제공된 정보 중에 고용허가서 신청 절차에 대한 내용은 없습니다. 다른 정보를 찾아보거나 전문가에게 문의해야 할 것 같습니다.


재입국 특례 제도를 신청하기 위한 조건은 다음과 같습니다:

1. 외국인근로자의 고용 등에 관한 법률 제18조의4 규정에 해당하는 자
    - 취업활동기간(5년 미만 또는 직권 연장 시 6년 미만) 중에 사업장을 변경하지 아니한 경우 또는 최초 근무한 업종과 동일 업종 내에서 사업장 변경하여 4년 10개월 근속한 경우(사용자와 취업활동기간 종료일까지의 근로계약 기간이 1년 이상일 것)
    - 휴업·폐업 등 외국인근로자의 책임이 아닌 사유로 사업장을 변경한 경우로서 사용자와 취업활동기간 종료일까지의 근로계약 기간이 1년 이상일 것 또는 근로계약 기간이 1년 미만이나 고용노동부(고용센터)가 재입국 특례 적용을 인정한 경우

2. 재입국하여 근로를 시작하는 날부터 효력이 발생하는 1년 이상의 근로계약을 체결하고 있을 것

3. 사업장별 외국인 고용한도 등 고용허가서 발급요건을 갖추고 있을 것 (다만, 내국인 구인노력은 요하지 않음)


죄송합니다, 제공된 정보에는 비전문취업 비자의 허용 업종 범위에 대한 구체적인 내용이 포함되어 있지 않습니다. 해당 정보를 확인하려면 관련 법률이나 정부 웹사이트를 참조하시는 것이 좋을 것 같습니다.


고용허가제 적용 대상 국가는 총 16개국으로, 태국, 필리핀, 스리랑카, 베트남, 인도네시아, 몽골, 파키스탄, 우즈베키스탄, 캄보디아, 중국, 방글라데시, 네팔, 미얀마, 키르기스스탄, 동티모르, 라오스입니다.


비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 다음과 같습니다:

